In [43]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from datetime import date
import os

In [44]:
PCA_per_TF = pd.read_csv('../exp/'+str(date.today())+'/PCA_results.csv')
#PCA_per_TF = PCA_per_TF.replace(['embryonic day 15.5','embryonic day 18.5','postnatal day 0.5','postnatal day 4','postnatal day 22','postnatal day 29'],[-5.5,-2.5,0.5,2,22,29])
PCA_per_TF = PCA_per_TF.set_index(['organ','dev_stage'])
PCA_per_TF.sort_index(inplace=True)


p_val = pd.DataFrame(index=PCA_per_TF.T.index, columns=['C(organ)','C(dev_stage)','C(organ):C(dev_stage)'])
for TF in p_val.index:
    test_data = pd.DataFrame(PCA_per_TF.loc[:,TF])
    test_data = test_data.reset_index()
    test_data.columns = ["organ", "dev_stage", "TF_expr"]
    data = test_data
    TF_lm = ols('TF_expr ~ C(organ)+C(dev_stage)+C(organ):C(dev_stage)', data=data).fit()
    table = sm.stats.anova_lm(TF_lm, typ=2)
    p_val.loc[TF,'C(organ)'] = table.iloc[0,3]
    p_val.loc[TF,'C(dev_stage)'] = table.iloc[1,3]
    p_val.loc[TF,'C(organ):C(dev_stage)'] = table.iloc[2,3]
    p_val.index.name = 'TF'
p_val

In [ ]:
try:
    p_val.to_csv('../exp/'+str(date.today())+'/p_vals.csv')
except FileNotFoundError:
    os.mkdir('../exp/'+str(date.today()))
    p_val.to_csv('../exp/'+str(date.today())+'/p_vals.csv')
